# Задание

1. Выберите набор данных (датасет) для решения задачи классификации или регрессии.
2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.
4. Обучите следующие модели:
    * одну из линейных моделей (линейную или полиномиальную регрессию при решении задачи регрессии, логистическую регрессию при решении задачи классификации);
    * SVM;
    * дерево решений.
5. Оцените качество моделей с помощью двух подходящих для задачи метрик. Сравните качество полученных моделей.
6. Постройте график, показывающий важность признаков в дереве решений.
7. Визуализируйте дерево решений или выведите правила дерева решений в текстовом виде.

В качестве набора данных используется dataset рейтингов университетов мира на основании трёх рейтингов. Датасет доступен по адресу: https://www.kaggle.com/mylesoneill/world-university-rankings

Из набора данных будет рассматриваться только файл cwurData.csv

## Основные характеристики набора данных

Подключаем все необходимые библиотеки

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib_inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline 
sns.set(style="ticks")
from io import StringIO

Подключаем Dataset

In [ ]:
data = pd.read_csv('cwurData.csv', sep=",")

Размер набора данных

In [ ]:
data.shape

Типы колонок

In [ ]:
data.dtypes

Проверяем, есть ли пропущенные значения

In [ ]:
data.isnull().sum()

Первые 5 строк датасета

In [ ]:
data.head()

In [ ]:
total_count = data.shape[0]
print('Всего строк: {}'.format(total_count))

Процент пропусков в // broad_impact

In [ ]:
(200 / 2200) * 100

Настройка отображения графиков

In [ ]:
# Задание формата графиков для сохранения высокого качества PNG
from IPython.display import set_matplotlib_formats
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")
# Задание ширины графиков, чтобы они помещались на A4
pd.set_option("display.width", 70)

## Обработка пропусков данных

### Очистка строк

Можно очистить строки, содержащие пропуски. При этом останутся данные только за 2014 и 2015 гг (см. описание датасета)

In [ ]:
# Удаление строк, содержащих пустые значения
data_no_null = data.dropna(axis=0, how='any')
(data.shape, data_no_null.shape)

Выведем первые 11 строк, чтобы убедиться, что данные в `national_rank` числовые (Jupyter Lab в предпросмотре CSV показывает не совсем верно)

In [ ]:
data_no_null.head(11)

In [ ]:
total_count = data_no_null.shape[0]
print('Всего строк: {}'.format(total_count))

## Кодирование категориальных признаков

Преобразуем названия стран, городов, и тд в числовые зеачения (label encoding)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
le = LabelEncoder()
    # "institution"
le.fit(data_no_null.institution.drop_duplicates()) 
data_no_null.institution = le.transform(data_no_null.institution)
    # "country"
le.fit(data_no_null["country"].drop_duplicates()) 
data_no_null["country"] = le.transform(data_no_null["country"])

Построим кореляционную матрицу

In [ ]:
ig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(data_no_null.corr(method='pearson'), ax=ax, annot=True, fmt='.3f')

## Предсказание целевого признака

Предскажем значение целевого признака world_rank по broad_impact и publications, поскольку их значения кореляции ближе всего к 1

### Разбиение выборки на обучающую и тестовую

In [ ]:
X = data_no_null[["broad_impact", "publications"]]
Y = data_no_null["world_rank"]
print('Входные данные:\n\n', X.head(), '\n\nВыходные данные:\n\n', Y.head())

Разделим выборку на обучающую и тестовую

In [ ]:
X_train,  X_test,  Y_train,  Y_test = train_test_split(X,  Y, random_state = 2022, test_size = 0.1)

Входные параметры обучающей выборки

In [ ]:
X_train.head()

Входные параметры тестовой выборки

In [ ]:
X_test.head()

Выходные параметры обучающей выборки

In [ ]:
Y_train.head()

Выходные параметры тестовой выборки

In [ ]:
Y_test.head()

### Построение линейной регрессии

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error,  median_absolute_error, r2_score

In [ ]:
Lin_Reg = LinearRegression().fit(X_train, Y_train)
lr_y_pred = Lin_Reg.predict(X_test)

Возьмем тот параметр, чья корреляция ближе всего к единице, т.е. broad_impact

In [ ]:
plt.scatter(X_test["broad_impact"], Y_test,    marker = 's', label = 'Тестовая выборка')
plt.scatter(X_test["broad_impact"], lr_y_pred, marker = 'o', label = 'Предсказанные данные')
plt.legend (loc = 'lower right')
plt.xlabel ('Рейтинг за широкое влияние')
plt.ylabel ('Целевой признак')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,  median_absolute_error, r2_score

In [ ]:
print('Средняя абсолютная ошибка:',   mean_absolute_error(Y_test, lr_y_pred))
print('Средняя квадратичная ошибка:', mean_squared_error(Y_test, lr_y_pred))
print('Median absolute error:',       median_absolute_error(Y_test, lr_y_pred))
print('Коэффициент детерминации:',    r2_score(Y_test, lr_y_pred))

### SVM

In [ ]:
from sklearn.svm import SVC , LinearSVC
from sklearn.datasets import make_blobs

In [ ]:
svc = SVC(kernel='linear')
svc.fit(X_train,Y_train)

In [ ]:
pred_y = svc.predict(X_test)

In [ ]:
plt.scatter(X_test["broad_impact"], Y_test, marker = 's', label = 'Тестовая выборка')
plt.scatter(X_test["broad_impact"], pred_y, marker = 'o', label = 'Предсказанные данные')
plt.legend (loc = 'lower right')
plt.xlabel ('рейтинг за широкое влияние')
plt.ylabel ('Целевой признак')
plt.show()

In [ ]:
print('Средняя абсолютная ошибка:',   mean_absolute_error(Y_test, pred_y))
print('Средняя квадратичная ошибка:', mean_squared_error(Y_test, pred_y))
print('Median absolute error:',       median_absolute_error(Y_test, pred_y))
print('Коэффициент детерминации:',    r2_score(Y_test, pred_y))

### Дерево (Tree)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.tree import export_graphviz
from sklearn import tree
import re
from IPython.core.display import HTML
from sklearn.tree import export_text
import graphviz
from IPython.display import Image
import pydotplus

Обучим дерево на всех признаках

In [ ]:
reg = tree.DecisionTreeRegressor()
reg = reg.fit(X_test, Y_test)

In [ ]:
pred_y = reg.predict(X_test)
plt.scatter(X_test["broad_impact"], Y_test,    marker = 's', label = 'Тестовая выборка')
plt.scatter(X_test["broad_impact"], pred_y, marker = 'o', label = 'Предсказанные данные')
plt.legend (loc = 'lower right')
plt.xlabel ('рейтинг за широкое влияние')
plt.ylabel ('Целевой признак')
plt.show()

#### Дерево в текстовом виде

In [ ]:
tree_rules = export_text(reg, feature_names=list(X.columns))
HTML('<pre>' + tree_rules + '</pre>')

#### Визуализация дерева

In [ ]:
# Визуализация дерева
def get_png_tree(tree_model_param, feature_names_param):
    dot_data = StringIO()
    export_graphviz(tree_model_param, out_file=dot_data, feature_names=feature_names_param,
                    filled=True, rounded=True, special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return graph.create_png()

In [ ]:
Image(get_png_tree(reg, X.columns), height='100%')

NameError: name 'Image' is not defined

#### Важность признаков в дереве

In [ ]:
from operator import itemgetter

def draw_feature_importances(tree_model, X_dataset, figsize=(18,5)):
    """
    Вывод важности признаков в виде графика
    """
    # Сортировка значений важности признаков по убыванию
    list_to_sort = list(zip(X_dataset.columns.values, tree_model.feature_importances_))
    sorted_list = sorted(list_to_sort, key=itemgetter(1), reverse = True)
    # Названия признаков
    labels = [x for x,_ in sorted_list]
    # Важности признаков
    data = [x for _,x in sorted_list]
    # Вывод графика
    fig, ax = plt.subplots(figsize=figsize)
    ind = np.arange(len(labels))
    plt.bar(ind, data)
    plt.xticks(ind, labels, rotation='vertical')
    # Вывод значений
    for a,b in zip(ind, data):
        plt.text(a-0.05, b+0.01, str(round(b,3)))
    plt.show()
    return labels, data

In [ ]:
boston_tree_regr_fl, boston_tree_regr_fd = draw_feature_importances(reg, X)